# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
dataset = datasets.fetch_california_housing()
# print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
print(dataset.keys()) # if you want to know what else is there in this dataset
dataset.target = dataset.target.astype(np.int) # so that we can classify
print(dataset.data.shape)
print(dataset.target.shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])
(20640, 8)
(20640,)


<ipython-input-2-8a8b10b52a90>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # Find the difference between features. Numpy automatically takes care of the size here.
  sq = diff*diff   # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample.

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.17205692108667528


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1699 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1686046511627907


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.3395999035912268


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that these two factors are balanced?

Answer for both nearest neighbour and random classifier.

# Answer to Question 1:
**1-Nearest Neighbor Classifier:** Increasing the percentage of the validation set will increase the accuracy of the validation set, but it will also reduce the amount of data available for training. This is because the 1-Nearest Neighbor Classifier is a very simple model that relies on memorizing the training data. With a larger validation set, the model will have less data to memorize for training, which can lead to a decrease in accuracy on the test set.

**Random Classifier:** Increasing the percentage of the validation set will also increase the accuracy of the validation set for a random classifier. However, this will not have a significant impact on the accuracy of the test set, as the random classifier is not a very good model and its predictions are essentially random.

**Reducing the percentage of the validation set will have the opposite effect for both classifiers.** It will decrease the accuracy of the validation set, but it will increase the amount of data available for training. This can lead to an increase in accuracy on the test set, especially for the 1-Nearest Neighbor Classifier.

# Answer to Question 2:
A larger train set and validation set will generally lead to better predictions of accuracy on the test set for both classifiers. This is because a larger train set allows the model to learn the underlying distribution of the data better, and a larger validation set allows the model to be evaluated more accurately.

However, it is important to note that the size of the train and validation sets is not the only factor that affects the accuracy of predictions on the test set. Other factors, such as the complexity of the model and the quality of the data, also play a role.

# Answer to Question 3:
A good percentage to reserve for the validation set is typically between 10% and 20% of the overall dataset. This provides a good balance between having enough data to evaluate the model accurately and having enough data to train the model effectively.

**In the case of a 1-Nearest Neighbor Classifier and Random Classifier evaluating a dataset whose 20% data is for the test set, and 25% of the remaining 80% data is used for the validation set, I would recommend reserving 10% of the overall dataset for the validation set.** This would leave 70% of the dataset for training and 20% for testing.

This percentage is appropriate because it is within the recommended range of 10% to 20%, and it provides a good balance between having enough data to evaluate the model accurately and having enough data to train the model effectively.

It is also important to note that the validation set should be representative of the test set in terms of its distribution of labels and features. This means that the validation set should be sampled from the same population as the test set.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [16]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ', Accuracy(testlabel, testpred))

Average validation accuracy is  0.3391130187195712
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module.

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# Answer to Question 1:
Yes, averaging the validation accuracy across multiple splits gives more consistent results. This is because it reduces the impact of any random variations in the training and validation sets. For example, if we have a small dataset, we may get a very different validation accuracy if we split it differently. However, if you average the validation accuracy across multiple splits, we will get a more robust estimate of the model's performance.

# Answer to Question 2:
Yes, averaging the validation accuracy across multiple splits also gives a more accurate estimate of test accuracy. This is because it reduces the overfitting bias. Overfitting is when the model learns the training data too well and is not able to generalize to new data. Averaging the validation accuracy across multiple splits helps to reduce overfitting by ensuring that the model is performing well on a variety of different splits of the data.


# Answer to Question 3:
Yes, we get a better estimate of the model's performance with higher iterations. This is because more iterations reduce the variance of the estimate. However, there is a trade-off between accuracy and computational cost. More iterations take longer to compute, so we need to find a balance between accuracy and speed.


# Answer to Question 4:
To some extent, we can deal with a very small train dataset or validation dataset by increasing the iterations. However, it is important to note that more iterations cannot compensate for a lack of data. If the dataset is too small, the model will not be able to learn the underlying patterns in the data, even with a large number of iterations.

In general, it is recommended to use a cross-validation procedure with at least 5 folds. This will give us a more reliable estimate of the model's performance. If you have a very small dataset, we may need to use a smaller number of folds or even leave-one-out cross-validation. However, it is important to be aware that this will give us a less accurate estimate of the model's performance.

Here are some additional tips for dealing with a very small train dataset or validation dataset:

* Use a simpler model. More complex models require more data to train.
* Use regularization techniques. Regularization techniques help to prevent the model from overfitting the training data.
* Use data augmentation techniques. Data augmentation techniques can be used to create more training data by artificially modifying existing data.